In [24]:
apis = {
    'esios':'datetime_utc',
    'redata':'datetime',
    'omie':'extraction_date_parsed',
    'open_meteo':'date'
    }

df_dates = None

from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()



StatementMeta(, af3b6f73-0f68-401e-a9e0-1f1a33723db2, 26, Finished, Available, Finished)

In [25]:

for api_name,date_col in apis.items():
    name_pattern = f"brz_{api_name}_"

    # Get tables from the Lakehouse catalog
    all_tables = spark.catalog.listTables()  # returns list of Table objects

    # Filter by prefix
    filtered_tables = [
        t for t in all_tables
        if t.name.startswith(name_pattern)
        and not t.name.endswith('mensual')
    ]
    # Select distinct dates for API
    for t in filtered_tables:
        df = spark.sql(f"SELECT DISTINCT {date_col} AS fact_date FROM {t.name}") \
         .withColumn("fact_date", F.col("fact_date").cast("date"))
        if df_dates is None:
            df_dates = df
        else:
            df_dates = df_dates.unionByName(df, allowMissingColumns=True)
    


StatementMeta(, af3b6f73-0f68-401e-a9e0-1f1a33723db2, 27, Finished, Available, Finished)

In [26]:

silver_lakehouse_id = "cab645cc-8a6f-49ec-a404-a83729660d6f"

# Spark session attached to Silver
spark_silver = (
    SparkSession.builder
    .appName("WriteToSilver")
    .config("spark.fabric.lakehouse.name", silver_lakehouse_id)
    .getOrCreate()
)

# Table name (schema.table)
silver_table_name = "lh_silver.slv_cod_date"

# Write DataFrame to Silver
(df_dates.write
    .format("delta")
    .mode("overwrite")  # or 'append'
    .option("overwriteSchema", "true")
    .saveAsTable(silver_table_name)  # <--- use saveAsTable
)

print(f"Dates saved at {silver_table_name}")

StatementMeta(, af3b6f73-0f68-401e-a9e0-1f1a33723db2, 28, Finished, Available, Finished)

Dates saved at lh_silver.slv_cod_date
